# Der Ruin des Spielers


In dieser Fallstudie studieren wir unsere Gewinnchancen beim Spielen im Casino. Insbesondere wollen wir am Ende folgende Frage beantworten können:
*Wenn wir mit einem bestimmten Anfangsbudget ins Casino gehen und unser Ziel ist, einen bestimmten Gewinn zu erzielen. In wie vielen Fällen gelingt uns dies, und in wie vielen Fällen sind wir zuvor ruiniert*.

Dabei werden wir sehen, wie uns Funktionen und Prozeduren helfen eine Aufgabe zu strukturieren. 
Zudem lernen wir auch, wie wir in einer Simulation Zufallszahlen nutzen können. 


### Zufallszahlen generieren

Jede Programmiersprache stellt uns Funktionalität zur Verfügung, um zufällige Zahlen zu generieren. Um zufällige Zahlen in Python zu generieren nutzen wir die Bibliothek ```random```, die wir aber zuerst importieren müssen. 

In [3]:
import random

Mithilfe der Funktion ```random.randint(a, b)``` können wir eine zufällige Ganzahl im Interval $[a, b]$ generieren. 
Das Modul stellt auch viele weitere Funktionen zur Verfügung, die Sie in der [Dokumentation](https://python.readthedocs.io/en/stable/library/random.html) finden. Wir brauchen hier nur diese eine Funktion. 

In [4]:
random.randint(0, 10)

0

Wir sehen, dass wenn wir diese Funktion mehrmals ausführen, dass jedes Mal ein anderer Wert erzeugt wird. Mittels einer Schleife, können wir so viele verschiedene, zufällige Werte generieren. 

In [5]:
i = 0
while i < 10:
    print(random.randint(0, 36))
    i += 1;


10
13
4
16
0
1
17
27
33
14


### Ein Spiel simulieren

Nun nutzen wir diese Zufallsfunktion um ein [Roulettespiel](https://de.wikipedia.org/wiki/Roulette) zu simulieren. Wir schreiben uns dafür eine Funktion. Diese bekommt als Argument unseren Einsatz, und gibt unseren Gewinn zurück. Der Gewinn ist zwei mal der Einsatz, oder 0 falls wir nicht gewinnen. Wir setzen dabei immer nur auf eine der Farben und schliessen nie andere Wetten ab.
Da es keinen Unterschied für die Gewinnchancen macht ob wir auf Rot oder Schwarz setzen, bilden wir dies in unserer Funktion nicht ab. Wir implementieren nur die Tatsache, dass wir in 19 von 37 Fällen verlieren und in 18 gewinnen. 

In [63]:
def play_game(bet): 
    random_number = random.randint(-1, 36)
    if random_number == 0:
        return 0
    elif random_number <= 18:
        return 0
    else: 
        return 2 * bet

In [67]:
play_game(5)

0

### Der Durchschnittliche Gewinn

Die vielleicht einfachste, aber dennoch interessante Frage ist, wieviel Geld wir im Durchschnitt gewinnen. Bei diesem einfachen Spiel könnten wir dies noch ohne Programm überprüfen. Bei komplexeren Spielen, ist es dann aber oft einfacher eine Simulation zu schreiben. Wir wollen nun `n` Spiele mit dem immer gleichen Einsatz `bet` simulieren.

In [68]:
def average_of_n_games(n, bet):
    cumulated_win = 0
    i = 0
    while i < n:
        cumulated_win = cumulated_win + play_game(bet)
        i = i + 1
    return cumulated_win / n

Wir können nun überprüfen, dass wenn ```n``` gross wird, wir immer einen Verlust einfahren werden. Dies ist der Gewinn des Casinos. Je grösser die Anzahl Spieler, desto stabiler der Gewinn. 

In [73]:
average_of_n_games(10000, 100)

95.72

### Spielen mit Zielbetrag

Statt auf ein einziges Spiel, fokussieren wir uns nun auf eine Serie von Spielen. Dabei verwenden wir die folgende Strategie:
- Wir spielen so lange weiter, bis wir einen bestimmten Zielbetrag erreicht haben.
- Wenn wir aber kein Geld mehr haben, dann müssen wir aufhören zu spielen.

Da wir schon die Funktion ```play_game``` geschrieben haben, können wir diese hier wiederverwenden. Wir nutzen diese Funktion als *black box*, d.h. wir müssen gar nicht wissen, wie die Funktion implementiert ist oder welches Spiel da eigentlich simuliert wird. Wichtig ist nur, dass wir wissen, wie wir die Funktion aufrufen und dass der Rückgabewert jeweils unser Gewinn oder Verlust ist.

Die folgende Funktion soll dann `True` zurück geben wenn wir das Ziel erreicht haben oder `False` wenn wir bankkrott sind.

In [74]:
def play_until_target_reached_or_ruined(initial_amount, bet, target_amount):
    balance = initial_amount

    while balance >= bet and balance < target_amount:
        balance = balance - bet
        balance = balance + play_game(bet)

    return balance >= target_amount

In [79]:
play_until_target_reached_or_ruined(100, 10, 110)

True

### Simulation - Wie viele Spiele gewinnen wir im Schnitt

Zum Schluss schreiben wir uns eine Funktion, die uns eine Anzahl Versuche simuliert und dabei zählt, wie oft wir unser Ziel erreichen und wie oft wir bankkrott gehen. Auch hier nutzen wir nun wieder die zuvor definierte Funktion.

In [80]:
def count_wins_in_n_games(number_of_tries, initial_amount, bet_per_game, target_amount):

    num_wins = 0

    i = 0
    while i < number_of_tries:
        if play_until_target_reached_or_ruined(initial_amount, bet_per_game, target_amount) == True:
            num_wins = num_wins + 1
        i += 1

    return num_wins

Nun können wir in der Simulation austesten, wie erfolgsversprechend unsere Strategie ist.

In [92]:
n_games = 1000

number_of_wins = count_wins_in_n_games(n_games, 100, 50, 150)
print("number of wins: ", number_of_wins)
print("number of ruins: ", n_games - number_of_wins)

number of wins:  608
number of ruins:  392


### Warnung

Haben wir tatsächlich eine Strategie gefunden, mit der wir das Casino überlisten können?

Um dies herauszufinden können wir den Gewinn und Verlust aus allen Spielen abschätzen und aufsummieren:

In [91]:
gain_per_game = 50 # target_amount - initial_amount
loss_per_game = 100 # initial_amount

overall_gain = number_of_wins * gain_per_game - (n_games-number_of_wins) * loss_per_game
print("overall gain: ", overall_gain)

overall gain:  -4000


Sie sehen, das Casino gewinnt immer. Sie können dort viel Spass haben, jedoch nicht zuverlässig Geld verdienen.